# IBM Data Science Capstone Project
## Final Report -- Neighborhood comparison
### Christopher Pouler

In [1]:
# Import libraries for below code

import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


---
## Content
---
    1. Introduction
        1.1 Background
        1.2 Problem
        1.3 Audience
    2. Data
        2.1 Required data
        2.2 Data sources
        2.3 Using the data
        2.4 Mapping data
    3. Methodology
        3.1 Process & strategy
        3.2 Methods (Machine learning, mapping, and exploratory)
    4. Results
        4.1 Results
    5. Discussion
        5.1 Discusssion, observations and recommendation for improvements
    6. Conclusion
        6.1 Decision taken & report conclusion
        
---
## Introduction

##### 1.1 Background
Studying economics, one of the areas of interest is real estate. Particularly relating to pricing structure in New York. Demand is fairly high, and price ceilings cause shortages. 
Housing has always been important to me, and having to decide where to live based on ammenities, cost, and location. 
I want to use the skills I've learned in this course to help evaluate housing in New York City, primarily Manhattan

##### 1.2 Problem
We want to try to find a possible rental location in Manhattan New York, NY.
We will try to do this by comparing the number of elligible apartments between Brooklyn, New York NY, and Manhattan, New York, NY 
Apartment must be:
* Rent no greater than $3000/ month
* Located in a neighborhood with similar ammenities and things to do as in Brooklyn
* I love Chinese food, so there must be at least one close Chinese food restaurant
* Must be within 1 mile of a subway stop

##### 1.3 Audience
The targeted audience is any individual living in New York City that is interested in finding an apartment

---
## Data

##### 2.1 Required Data
Use Foursquare to identify venues around New York Cty. We'll need lists of neighborhoods in New York City with their geodata.
We'll also need a list of beds, price , location, address to help compare the apartments. If this information is not available in Foursquare, we will need to get it from listing sites.

##### 2.2 Data Sources
In order to make a good choice of a similar apartment in Manhattan NY, the following data is required: List/Information on neighborhoods form Manhattan with their Geodata ( latitud and longitud. List/Information about the subway metro stations in Manhattan with geodata. Listed apartments for rent in Manhattan area with descriptions ( how many beds, price, location, address) Venues and ammenities in the Manhattan neighborhoods (e.g. top 10) 2.3 sources and manipulation The list of Manhattan neighborhoods is worked out during Lab exercise during the course. 

A list of places for rent was collected by web-browsing real estate companies in Manhattan : 
* http://www.rentmanhattan.com/index.cfm?page=search&state=results
* https://www.nestpick.com/search?city=new-york&page=1&order=relevance&district=manhattan&gclid=CjwKCAiAjNjgBRAgEiwAGLlf2hkP3A-cPxjZYkURqQEswQK2jKQEpv_MvKcrIhRWRzNkc_r-fGi0lxoCA7cQAvD_BwE&type=apartment&display=list
* https://www.realtor.com/apartments/Manhattan_NY

Longitude and Latitude data were derrived via Foursquare and a procedure we did in the lab

##### 2.3 Using the data
The data will be used as follows: Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB) Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations. create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area. Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim. Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.

##### 2.4 Mapping Data
The following maps were created to facilitate the analysis and the choice of the palace to live. Manhattan map of Neighborhoods manhattan subway metro locations Manhattan map of places for rent Manhattan map of clustered venues and neighborhoods Combined maps of Manhattan rent places with subway locations Combined maps of Manhattan rent places with subway locations and venues clusters

---
## Methodology
##### 3.1 Process & strategy
The strategy is based on mapping the above described data in section 2.0, in order to facilitate the choice of at least two candidate places for rent. The choice is made based on the demands imposed : location near a subway, rental price and similar venues to Singapore. This visual approach and maps with popups labels allow quick identification of location, price and feature, thus making the selection very easy.

The procesing of these DATA and its mapping will allow to answer the key questions to make a decision:

what is the cost of available rental places that meet the demands?
what is the cost of rent around a mile radius from each subway metro station?
what is the area of Manhattan with best rental pricing that meets criteria established?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.

##### 3.2 Methods (Machine learning, mapping, and exploratory)
Find close restaurants and subway stops that are within the search criteria
Map them to create a visual component of how close they are
Select a neighborhood in Manhattan that has the best ammenities and things to do (arbitrary)
Find apartments based on price criteria searching websites listed above
Find close restaurants and subway stops to apartment that fit identical search criteria
Map the restaurants and subway stops to see if they fit within benchmark case
Select apartment based on lowest rent, most restaurants nearby, and proximity to the metro


###### Our Benchmark appartment is 1230 Avenue Y, Brroklyn, NY
Lets start by finding some restaurants nearby our benchmark apartment

In [2]:
address = '1230 Avenue Y, Brooklyn, NY 11235'
geolocator = Nominatim()
location = geolocator.geocode(address)
lat_bm = location.latitude
long_bm = location.longitude
print('The geograpical coordinate of the benchmark location is {}, {}.'.format(lat_bm, long_bm))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of the benchmark location is 40.5893636, -73.9574884723781.


In [3]:
lat_bm = 40.5893636
long_bm = -73.9574884723781

In [4]:
# @hidden_cell
CLIENT_ID = 'P0WD2Z2MBC3IB5A2DZLVYNITFAUNPPIHOFPSQU35BV330GDC' # your Foursquare ID
CLIENT_SECRET = 'SJX3AIENSBHNBDLCN4W5PP5RXPZSH30FLKC0QNQBJ0KGMXPO' # your Foursquare Secret
VERSION = '20180604'

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [5]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Chinese' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_bm, 
    long_bm, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=P0WD2Z2MBC3IB5A2DZLVYNITFAUNPPIHOFPSQU35BV330GDC&client_secret=SJX3AIENSBHNBDLCN4W5PP5RXPZSH30FLKC0QNQBJ0KGMXPO&ll=40.5893636,-73.9574884723781&v=20180604&query=Chinese&radius=500&limit=30'

In [6]:
# results display is hidden for report simplification 
results_bm_cf = requests.get(url).json()
#results

In [7]:
# assign relevant part of JSON to venues
venues_bm_cf = results_bm_cf['response']['venues']

# tranform venues into a dataframe
dataframe_bm_cf = json_normalize(venues_bm_cf)
dataframe_bm_cf.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",False,4f9869f5e4b037b568f0526d,13th Ave,US,Brooklyn,United States,540,"[13th Ave, Brooklyn, NY 11214, United States]","[{'label': 'display', 'lat': 40.59421807714243...",40.594218,-73.957476,11214,NY,Chinese Food,v-1555600604
1,"[{'shortName': 'Mexican', 'pluralName': 'Mexic...",False,4c1fb5cd63750f47918dbb67,1508 Sheepshead Bay Rd,US,Brooklyn,United States,406,"[1508 Sheepshead Bay Rd, Brooklyn, NY 11235, U...","[{'label': 'display', 'lat': 40.58728943729068...",40.587289,-73.953527,11235,NY,Yummy Chinese and Taco,v-1555600604


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_bm_cf = ['name', 'categories'] + [col for col in dataframe_bm_cf.columns if col.startswith('location.')] + ['id']
dataframe_filtered_bm_cf = dataframe_bm_cf.loc[:, filtered_columns_bm_cf]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_bm_cf['categories'] = dataframe_filtered_bm_cf.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_bm_cf.columns = [column.split('.')[-1] for column in dataframe_filtered_bm_cf.columns]

dataframe_filtered_bm_cf.head(10)

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Chinese Food,Chinese Restaurant,13th Ave,US,Brooklyn,United States,540,"[13th Ave, Brooklyn, NY 11214, United States]","[{'label': 'display', 'lat': 40.59421807714243...",40.594218,-73.957476,11214,NY,4f9869f5e4b037b568f0526d
1,Yummy Chinese and Taco,Mexican Restaurant,1508 Sheepshead Bay Rd,US,Brooklyn,United States,406,"[1508 Sheepshead Bay Rd, Brooklyn, NY 11235, U...","[{'label': 'display', 'lat': 40.58728943729068...",40.587289,-73.953527,11235,NY,4c1fb5cd63750f47918dbb67


#### Now lets look for a nearby subway stop


In [9]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Subway Metro Station' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_bm, 
    long_bm, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
# results display is hidden for report simplification 
results_bm_sub = requests.get(url).json()

# assign relevant part of JSON to venues
venues_bm_sub = results_bm_sub['response']['venues']
# tranform venues into a dataframe
dataframe_bm_sub = json_normalize(venues_bm_sub)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_bm_sub = ['name', 'categories'] + [col for col in dataframe_bm_sub.columns if col.startswith('location.')] + ['id']
dataframe_filtered_bm_sub = dataframe_bm_sub.loc[:, filtered_columns_bm_sub]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered_bm_sub['categories'] = dataframe_filtered_bm_sub.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered_bm_sub.columns = [column.split('.')[-1] for column in dataframe_filtered_bm_sub.columns]

dataframe_filtered_bm_sub.head(10)

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MTA Regional Bus & Subway Tranfer - Sheepshead...,Metro Station,NaN,US,NaN,United States,332,"[New York, United States]","[{'label': 'display', 'lat': 40.58751537429961...",40.587515,-73.954394,NaN,New York,4e7f77be0aafd3ac87f11499
1,MTA Subway - Sheepshead Bay (B/Q),Metro Station,1432 Sheepshead Bay Rd,US,Brooklyn,United States,365,"[1432 Sheepshead Bay Rd, Brooklyn, NY 11235, U...","[{'label': 'display', 'lat': 40.58713410088744...",40.587134,-73.954308,11235,NY,4aff3636f964a520ed3522e3


In [10]:
benchmark_map = folium.Map(location=[lat_bm, long_bm], zoom_start=15) 

folium.features.CircleMarker(
    [lat_bm, long_bm],
    radius=10,
    color='red',
    popup='Benchmark Apartment',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(benchmark_map)

# add the chinese restaurant as blue circle markers
for lat, lng, label in zip(dataframe_filtered_bm_cf.lat, dataframe_filtered_bm_cf.lng, dataframe_filtered_bm_cf.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(benchmark_map)

    # add the subway as blue circle markers
for lat, lng, label in zip(dataframe_filtered_bm_sub.lat, dataframe_filtered_bm_sub.lng, dataframe_filtered_bm_sub.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(benchmark_map)
    
#benchmark_map    

##### Summary
We see at least two subway stops within 500 meters of the benchmark apartment. This is within the 1 mile requirement in the problem statement. Any apartment in Manhattan must meet similar criteria.

Additionally, there are two chinese restaurants within 500 meters of the benchmark apartment. Any apartment we look for must have a similar listing of restaurants.


---
##### End of benchmarking phase
##### Now lets try to find out which neighborhood we want to live in.
---


In [11]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [12]:
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


Data downloaded!


In [13]:
address = 'Manhattan, New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
lat_manhattan = location.latitude
long_manhattan = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(lat_manhattan, long_manhattan))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [14]:
lat_manhattan = 40.7900869
long_manhattan = -73.9598295

In [15]:
# Get a list of every Manhattan neighborhood
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head(100)


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [16]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [17]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [20]:
results = requests.get(url).json()

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


##### Find out which neighborhood in Manhattan has the best ammenities to find an appartment in

In [25]:
print(manhattan_venues.shape)

(3326, 7)


In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5
for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.07
1                Park  0.07
2               Hotel  0.05
3                 Gym  0.04
4  Italian Restaurant  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1     Coffee Shop  0.05
2             Bar  0.04
3  Cosmetics Shop  0.04
4            Café  0.04


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1       Cosmetics Shop  0.07
2   Chinese Restaurant  0.05
3  American Restaurant  0.05
4   Seafood Restaurant  0.05


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.10
1     Dim Sum Restaurant  0.04
2    American Restaurant  0.04
3           Cocktail Bar  0.04
4  Vietnamese Restaurant  0.04


----Civic Center----
                  venue 

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Wine Shop,Clothing Store,Italian Restaurant,Shopping Mall,Fountain,Men's Store
1,Carnegie Hill,Pizza Place,Coffee Shop,Bar,Café,Cosmetics Shop,Bookstore,Spa,French Restaurant,Grocery Store,Yoga Studio
2,Central Harlem,African Restaurant,Cosmetics Shop,French Restaurant,American Restaurant,Chinese Restaurant,Art Gallery,Gym / Fitness Center,Seafood Restaurant,Bookstore,Beer Bar
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,Hotel,Seafood Restaurant,Theater,American Restaurant,Health & Beauty Service
4,Chinatown,Chinese Restaurant,American Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Cocktail Bar,Noodle House,Hotpot Restaurant,Ice Cream Shop,Bubble Tea Shop,Bar
5,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,Coffee Shop,French Restaurant,Yoga Studio,Cocktail Bar,Sandwich Place,Park,Sporting Goods Shop
6,Clinton,Theater,Hotel,Gym / Fitness Center,Italian Restaurant,American Restaurant,Wine Shop,Spa,Gym,Food Court,Cocktail Bar
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Pizza Place,French Restaurant,Steakhouse,Street Art,Café
8,East Village,Bar,Wine Bar,Mexican Restaurant,Ice Cream Shop,Cocktail Bar,Pizza Place,Chinese Restaurant,Coffee Shop,Ramen Restaurant,Vegetarian / Vegan Restaurant
9,Financial District,Coffee Shop,Hotel,Steakhouse,Wine Shop,Gym,Bar,Italian Restaurant,Park,Pizza Place,Food Truck


##### Results
Based on this list of the most frequent ammenities in each neighborhood. I've chosen the Upper West Side to base my search on. This helps narrow my appartment search even further. Based on rent, proximity to the subway, and proximity to restaurants I'll decide my final location

In [29]:
manhattan_data[12:13]

,Borough,Neighborhood,Latitude,Longitude
12,Manhattan,Upper West Side,40.787658,-73.977059


In [30]:
lat_UWS = 40.787658 
long_UWS = -73.977059
print('The geograpical coordinate of the Upper West Side are {}, {}.'.format(lat_UWS, long_UWS))

The geograpical coordinate of the Upper West Side are 40.787658, -73.977059.


##### Based on searching websites for apartments, we have found two potential locations:

   
    Apartment     Address              Cost
    A             250 West 105th St.   $2,485
B             170 West 109th St.   $2,750

Lets focus on apartment A for now
    

In [31]:
address = '250 West 105th St., Manhattan, NY 10025'
geolocator = Nominatim()
location = geolocator.geocode(address)
lat_apt_a = location.latitude
long_apt_a = location.longitude
print('The geograpical coordinate of the first apartment location is {}, {}.'.format(lat_apt_a, long_apt_a))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of the first apartment location is 40.80058375, -73.9686161026142.


In [32]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Chinese' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_apt_a, 
    long_apt_a, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=P0WD2Z2MBC3IB5A2DZLVYNITFAUNPPIHOFPSQU35BV330GDC&client_secret=SJX3AIENSBHNBDLCN4W5PP5RXPZSH30FLKC0QNQBJ0KGMXPO&ll=40.80058375,-73.9686161026142&v=20180604&query=Chinese&radius=500&limit=30'

In [33]:
# results display is hidden for report simplification 
results_aptA_CF = requests.get(url).json()

In [34]:
# assign relevant part of JSON to venues
venues_aptA_CF = results_aptA_CF['response']['venues']

# tranform venues into a dataframe
dataframe_aptA_CF = json_normalize(venues_aptA_CF)
dataframe_aptA_CF.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b885bd3f964a520dbf131e3,917 Columbus Ave,US,New York,United States,105th St.,493,"[917 Columbus Ave (105th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.79855362787655...",40.798554,-73.963403,10025,NY,U Like Chinese,v-1555600637
1,"[{'shortName': 'Private School', 'pluralName':...",NaN,NaN,NaN,NaN,NaN,NaN,False,55d215bd498e3ddc5b233fb7,Broadway,US,New York,United States,103rd,164,"[Broadway (103rd), New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.79911042808877...",40.799110,-73.968480,10025,NY,Upper West Side Chinese Tutoring,v-1555600637
2,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ca9c9a444a8224bd9842440,Rama 23,US,New York,United States,NaN,270,"[Rama 23, New York, NY 10025, United States]","[{'label': 'display', 'lat': 40.800832, 'lng':...",40.800832,-73.965421,10025,NY,Rama chinese food,v-1555600637
3,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f43e7c519836ed001912bf3,917 Columbus Ave,US,New York,United States,NaN,500,"[917 Columbus Ave, New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.798417, 'lng':...",40.798417,-73.963407,10025,NY,Hwa Ying Chinese Restaurant,v-1555600637
4,"[{'shortName': 'Gym', 'pluralName': 'Gyms', 'i...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c4508248c1f20a11a553d99,2726 Broadway,US,New York,United States,NaN,76,"[2726 Broadway, New York, NY 10025, United Sta...","[{'label': 'display', 'lat': 40.800064317067, ...",40.800064,-73.968024,10025,NY,New Amsterdam Fencing Academy,v-1555600637


In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_aptA_CF = ['name', 'categories'] + [col for col in dataframe_aptA_CF.columns if col.startswith('location.')] + ['id']
dataframe_filtered_aptA_CF = dataframe_aptA_CF.loc[:, filtered_columns_aptA_CF]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_aptA_CF['categories'] = dataframe_filtered_aptA_CF.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_aptA_CF.columns = [column.split('.')[-1] for column in dataframe_filtered_aptA_CF.columns]

dataframe_filtered_aptA_CF.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,U Like Chinese,Chinese Restaurant,917 Columbus Ave,US,New York,United States,105th St.,493,"[917 Columbus Ave (105th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.79855362787655...",40.798554,-73.963403,10025,NY,4b885bd3f964a520dbf131e3
1,Upper West Side Chinese Tutoring,Private School,Broadway,US,New York,United States,103rd,164,"[Broadway (103rd), New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.79911042808877...",40.799110,-73.968480,10025,NY,55d215bd498e3ddc5b233fb7
2,Rama chinese food,Chinese Restaurant,Rama 23,US,New York,United States,NaN,270,"[Rama 23, New York, NY 10025, United States]","[{'label': 'display', 'lat': 40.800832, 'lng':...",40.800832,-73.965421,10025,NY,4ca9c9a444a8224bd9842440
3,Hwa Ying Chinese Restaurant,Chinese Restaurant,917 Columbus Ave,US,New York,United States,NaN,500,"[917 Columbus Ave, New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.798417, 'lng':...",40.798417,-73.963407,10025,NY,4f43e7c519836ed001912bf3
4,New Amsterdam Fencing Academy,Gym,2726 Broadway,US,New York,United States,NaN,76,"[2726 Broadway, New York, NY 10025, United Sta...","[{'label': 'display', 'lat': 40.800064317067, ...",40.800064,-73.968024,10025,NY,4c4508248c1f20a11a553d99
5,Pearls,Chinese Restaurant,796 Amsterdam Ave,US,New York,United States,W 99th St,531,"[796 Amsterdam Ave (W 99th St), New York, NY 1...","[{'label': 'display', 'lat': 40.79581795808015...",40.795818,-73.969046,10025,NY,4a038994f964a520c5711fe3
6,Concord Garden,Chinese Restaurant,990 Amsterdam Ave,US,New York,United States,NaN,409,"[990 Amsterdam Ave, New York, NY 10025, United...","[{'label': 'display', 'lat': 40.8023383729882,...",40.802338,-73.964338,10025,NY,4be1f4f5edbb0f471449a715


##### Results
Apartment A has plenty of chinese food resaurants close by. It satisfies this criteria of the problem

Below is the code to map this

In [36]:
aptA_food_map = folium.Map(location=[lat_apt_a, long_apt_a], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent apartment A
folium.features.CircleMarker(
    [lat_apt_a, long_apt_a],
    radius=10,
    color='red',
    popup='Apartment A',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(aptA_food_map)

# add the food as blue circle markers
for lat, lng, label in zip(dataframe_filtered_aptA_CF.lat, dataframe_filtered_aptA_CF.lng, dataframe_filtered_aptA_CF.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(aptA_food_map)

# display map
#aptA_food_map

##### Now lets look for a metro station stop

In [37]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Subway Metro Station' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_apt_a, 
    long_apt_a, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
# results display is hidden for report simplification 
results_apta_sub = requests.get(url).json()

# assign relevant part of JSON to venues
venues_apta_sub = results_apta_sub['response']['venues']
# tranform venues into a dataframe
dataframe_apta_sub = json_normalize(venues_apta_sub)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_apta_sub = ['name', 'categories'] + [col for col in dataframe_apta_sub.columns if col.startswith('location.')] + ['id']
dataframe_filtered_apta_sub = dataframe_apta_sub.loc[:, filtered_columns_apta_sub]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered_apta_sub['categories'] = dataframe_filtered_apta_sub.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered_apta_sub.columns = [column.split('.')[-1] for column in dataframe_filtered_apta_sub.columns]

dataframe_filtered_apta_sub.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MTA Subway - Cathedral Pkwy/110th St (1),Metro Station,Cathedral Pkwy,US,New York,United States,at Broadway,448,"[Cathedral Pkwy (at Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.80434394152203...",40.804344,-73.966704,10025,NY,4b5b9b17f964a520720a29e3
1,MTA Subway - 103rd St (1),Metro Station,W 103rd St.,US,New York,United States,at Broadway,143,"[W 103rd St. (at Broadway), New York, NY 10029...","[{'label': 'display', 'lat': 40.79930941695626...",40.799309,-73.968397,10029,NY,4b187f73f964a52086d323e3


##### Results
Apartment A has a few subway stops nearby. It satisfies this criteria of the problem.

Below is the code to map this

In [38]:
aptA_subway_map = folium.Map(location=[lat_apt_a, long_apt_a], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [lat_apt_a, long_apt_a],
    radius=10,
    color='red',
    popup='Apartment A',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(aptA_subway_map)

# add the subway as blue circle markers
for lat, lng, label in zip(dataframe_filtered_apta_sub.lat, dataframe_filtered_apta_sub.lng, dataframe_filtered_apta_sub.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(aptA_subway_map)

# display map
#aptA_subway_map

In [39]:
apt_A_map = folium.Map(location=[lat_apt_a, long_apt_a], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent apartment A
folium.features.CircleMarker(
    [lat_apt_a, long_apt_a],
    radius=10,
    color='red',
    popup='Apartment A',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(apt_A_map)

# add the food as blue circle markers
for lat, lng, label in zip(dataframe_filtered_aptA_CF.lat, dataframe_filtered_aptA_CF.lng, dataframe_filtered_aptA_CF.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(apt_A_map)
    
# add the subway as green circle markers
for lat, lng, label in zip(dataframe_filtered_apta_sub.lat, dataframe_filtered_apta_sub.lng, dataframe_filtered_apta_sub.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(apt_A_map)
    
# display map
#apt_A_map

In [40]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_aptA_CF = ['name', 'categories'] + [col for col in dataframe_aptA_CF.columns if col.startswith('location.')] + ['id']
dataframe_filtered_aptA_CF = dataframe_aptA_CF.loc[:, filtered_columns_aptA_CF]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_aptA_CF['categories'] = dataframe_filtered_aptA_CF.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_aptA_CF.columns = [column.split('.')[-1] for column in dataframe_filtered_aptA_CF.columns]

dataframe_filtered_aptA_CF.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,U Like Chinese,Chinese Restaurant,917 Columbus Ave,US,New York,United States,105th St.,493,"[917 Columbus Ave (105th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.79855362787655...",40.798554,-73.963403,10025,NY,4b885bd3f964a520dbf131e3
1,Upper West Side Chinese Tutoring,Private School,Broadway,US,New York,United States,103rd,164,"[Broadway (103rd), New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.79911042808877...",40.799110,-73.968480,10025,NY,55d215bd498e3ddc5b233fb7
2,Rama chinese food,Chinese Restaurant,Rama 23,US,New York,United States,NaN,270,"[Rama 23, New York, NY 10025, United States]","[{'label': 'display', 'lat': 40.800832, 'lng':...",40.800832,-73.965421,10025,NY,4ca9c9a444a8224bd9842440
3,Hwa Ying Chinese Restaurant,Chinese Restaurant,917 Columbus Ave,US,New York,United States,NaN,500,"[917 Columbus Ave, New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.798417, 'lng':...",40.798417,-73.963407,10025,NY,4f43e7c519836ed001912bf3
4,New Amsterdam Fencing Academy,Gym,2726 Broadway,US,New York,United States,NaN,76,"[2726 Broadway, New York, NY 10025, United Sta...","[{'label': 'display', 'lat': 40.800064317067, ...",40.800064,-73.968024,10025,NY,4c4508248c1f20a11a553d99
5,Pearls,Chinese Restaurant,796 Amsterdam Ave,US,New York,United States,W 99th St,531,"[796 Amsterdam Ave (W 99th St), New York, NY 1...","[{'label': 'display', 'lat': 40.79581795808015...",40.795818,-73.969046,10025,NY,4a038994f964a520c5711fe3
6,Concord Garden,Chinese Restaurant,990 Amsterdam Ave,US,New York,United States,NaN,409,"[990 Amsterdam Ave, New York, NY 10025, United...","[{'label': 'display', 'lat': 40.8023383729882,...",40.802338,-73.964338,10025,NY,4be1f4f5edbb0f471449a715


---
---
# ----------------------------------
---
---
#### Time to look at apartment B

In [41]:
address = '170 West 109th St., Manhattan, NY 10025'
geolocator = Nominatim()
location = geolocator.geocode(address)
lat_apt_b = location.latitude
long_apt_b = location.longitude
print('The geograpical coordinate of the first apartment location is {}, {}.'.format(lat_apt_b, long_apt_b))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of the first apartment location is 40.8019381, -73.9638294167164.


In [42]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Chinese' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_apt_b, 
    long_apt_b, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=P0WD2Z2MBC3IB5A2DZLVYNITFAUNPPIHOFPSQU35BV330GDC&client_secret=SJX3AIENSBHNBDLCN4W5PP5RXPZSH30FLKC0QNQBJ0KGMXPO&ll=40.8019381,-73.9638294167164&v=20180604&query=Chinese&radius=500&limit=30'

In [43]:
# results display is hidden for report simplification 
results_aptb_CF = requests.get(url).json()

In [44]:
# assign relevant part of JSON to venues
venues_aptb_CF = results_aptb_CF['response']['venues']

# tranform venues into a dataframe
dataframe_aptb_CF = json_normalize(venues_aptb_CF)
dataframe_aptb_CF.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",False,4b885bd3f964a520dbf131e3,917 Columbus Ave,US,New York,United States,105th St.,378,"[917 Columbus Ave (105th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.79855362787655...",40.798554,-73.963403,10025,NY,U Like Chinese,v-1555600638
1,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",False,4ca9c9a444a8224bd9842440,Rama 23,US,New York,United States,NaN,182,"[Rama 23, New York, NY 10025, United States]","[{'label': 'display', 'lat': 40.800832, 'lng':...",40.800832,-73.965421,10025,NY,Rama chinese food,v-1555600638
2,"[{'shortName': 'Chinese', 'pluralName': 'Chine...",False,4f43e7c519836ed001912bf3,917 Columbus Ave,US,New York,United States,NaN,393,"[917 Columbus Ave, New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.798417, 'lng':...",40.798417,-73.963407,10025,NY,Hwa Ying Chinese Restaurant,v-1555600638
3,"[{'shortName': 'Private School', 'pluralName':...",False,55d215bd498e3ddc5b233fb7,Broadway,US,New York,United States,103rd,502,"[Broadway (103rd), New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.79911042808877...",40.799110,-73.968480,10025,NY,Upper West Side Chinese Tutoring,v-1555600638
4,"[{'shortName': 'Food Truck', 'pluralName': 'Fo...",False,5772bb62498e562ece7193c8,Broadway,US,New York,United States,116 St,546,"[Broadway (116 St), New York, NY 10027, United...","[{'label': 'display', 'lat': 40.80676400036688...",40.806764,-73.964996,10027,NY,Healthy Corner Chinese Food Cart,v-1555600638


In [45]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_aptb_CF = ['name', 'categories'] + [col for col in dataframe_aptb_CF.columns if col.startswith('location.')] + ['id']
dataframe_filtered_aptb_CF = dataframe_aptb_CF.loc[:, filtered_columns_aptb_CF]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_aptb_CF['categories'] = dataframe_filtered_aptb_CF.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_aptb_CF.columns = [column.split('.')[-1] for column in dataframe_filtered_aptb_CF.columns]

dataframe_filtered_aptb_CF.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,U Like Chinese,Chinese Restaurant,917 Columbus Ave,US,New York,United States,105th St.,378,"[917 Columbus Ave (105th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.79855362787655...",40.798554,-73.963403,10025,NY,4b885bd3f964a520dbf131e3
1,Rama chinese food,Chinese Restaurant,Rama 23,US,New York,United States,NaN,182,"[Rama 23, New York, NY 10025, United States]","[{'label': 'display', 'lat': 40.800832, 'lng':...",40.800832,-73.965421,10025,NY,4ca9c9a444a8224bd9842440
2,Hwa Ying Chinese Restaurant,Chinese Restaurant,917 Columbus Ave,US,New York,United States,NaN,393,"[917 Columbus Ave, New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.798417, 'lng':...",40.798417,-73.963407,10025,NY,4f43e7c519836ed001912bf3
3,Upper West Side Chinese Tutoring,Private School,Broadway,US,New York,United States,103rd,502,"[Broadway (103rd), New York, NY 10025, United ...","[{'label': 'display', 'lat': 40.79911042808877...",40.799110,-73.968480,10025,NY,55d215bd498e3ddc5b233fb7
4,Healthy Corner Chinese Food Cart,Food Truck,Broadway,US,New York,United States,116 St,546,"[Broadway (116 St), New York, NY 10027, United...","[{'label': 'display', 'lat': 40.80676400036688...",40.806764,-73.964996,10027,NY,5772bb62498e562ece7193c8
5,Concord Garden,Chinese Restaurant,990 Amsterdam Ave,US,New York,United States,NaN,61,"[990 Amsterdam Ave, New York, NY 10025, United...","[{'label': 'display', 'lat': 40.8023383729882,...",40.802338,-73.964338,10025,NY,4be1f4f5edbb0f471449a715
6,New Amsterdam Fencing Academy,Gym,2726 Broadway,US,New York,United States,NaN,410,"[2726 Broadway, New York, NY 10025, United Sta...","[{'label': 'display', 'lat': 40.800064317067, ...",40.800064,-73.968024,10025,NY,4c4508248c1f20a11a553d99


##### Results
Apartment B has plenty of chinese food resaurants close by. It satisfies this criteria of the problem

Below is the code to map this

In [46]:
aptb_food_map = folium.Map(location=[lat_apt_b, long_apt_b], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent apartment A
folium.features.CircleMarker(
    [lat_apt_b, long_apt_b],
    radius=10,
    color='red',
    popup='Apartment B',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(aptb_food_map)

# add the food as blue circle markers
for lat, lng, label in zip(dataframe_filtered_aptb_CF.lat, dataframe_filtered_aptb_CF.lng, dataframe_filtered_aptb_CF.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(aptb_food_map)

# display map
#aptb_food_map

##### Now lets look for a metro station stop

In [47]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
search_query = 'Subway Metro Station' #Add a search requirement for restaurants
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat_apt_b, 
    long_apt_b, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
# results display is hidden for report simplification 
results_aptb_sub = requests.get(url).json()

# assign relevant part of JSON to venues
venues_aptb_sub = results_aptb_sub['response']['venues']
# tranform venues into a dataframe
dataframe_aptb_sub = json_normalize(venues_aptb_sub)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns_aptb_sub = ['name', 'categories'] + [col for col in dataframe_aptb_sub.columns if col.startswith('location.')] + ['id']
dataframe_filtered_aptb_sub = dataframe_aptb_sub.loc[:, filtered_columns_aptb_sub]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered_aptb_sub['categories'] = dataframe_filtered_aptb_sub.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered_aptb_sub.columns = [column.split('.')[-1] for column in dataframe_filtered_aptb_sub.columns]

dataframe_filtered_aptb_sub.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MTA Subway - Cathedral Pkwy/110th St (1),Metro Station,Cathedral Pkwy,US,New York,United States,at Broadway,361,"[Cathedral Pkwy (at Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.80434394152203...",40.804344,-73.966704,10025,NY,4b5b9b17f964a520720a29e3
1,MTA Subway - 103rd St (1),Metro Station,W 103rd St.,US,New York,United States,at Broadway,483,"[W 103rd St. (at Broadway), New York, NY 10029...","[{'label': 'display', 'lat': 40.79930941695626...",40.799309,-73.968397,10029,NY,4b187f73f964a52086d323e3
2,MTA Subway - 110th St/Cathedral Parkway (B/C),Metro Station,110th St,US,New York,United States,8th Ave / Frederick Douglass Blvd,420,"[110th St (8th Ave / Frederick Douglass Blvd),...","[{'label': 'display', 'lat': 40.80030913025532...",40.800309,-73.959321,10026,NY,4b65168ff964a52091e32ae3


##### Results
Apartment B has a few subway stops nearby. It satisfies this criteria of the problem.

Below is the code to map this

In [48]:
aptb_subway_map = folium.Map(location=[lat_apt_b, long_apt_b], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [lat_apt_b, long_apt_b],
    radius=10,
    color='red',
    popup='Apartment B',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(aptb_subway_map)

# add the subway as blue circle markers
for lat, lng, label in zip(dataframe_filtered_aptb_sub.lat, dataframe_filtered_aptb_sub.lng, dataframe_filtered_aptb_sub.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(aptb_subway_map)

# display map
#aptb_subway_map

In [49]:
apt_b_map = folium.Map(location=[lat_apt_b, long_apt_b], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent apartment A
folium.features.CircleMarker(
    [lat_apt_b, long_apt_b],
    radius=10,
    color='red',
    popup='Apartment B',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(apt_b_map)

# add the food as blue circle markers
for lat, lng, label in zip(dataframe_filtered_aptb_CF.lat, dataframe_filtered_aptb_CF.lng, dataframe_filtered_aptb_CF.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(apt_b_map)
    
# add the subway as green circle markers
for lat, lng, label in zip(dataframe_filtered_aptb_sub.lat, dataframe_filtered_aptb_sub.lng, dataframe_filtered_aptb_sub.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(apt_b_map)
    
# display map
#apt_b_map


---
## 4. Results
##### 4.1 Results
We've found two apartments that satisfy all three criteria for our problem. We were looking for an apartment that has rent cheaper than $3,000, close to a chinese restaurant, and close to a subway stop.

Using Foursquare, we searched the area near each apartment, and are able to map each location. I have the benchmark, apartment A, and apartment B all listed below. The subway stops are listed in green, and resaurants are shown in blue. Please take some time to review these maps.

Please recall the cost of apartment A and apartment B as well. This will also factor into our decision about which apartment is better.

        Apartment     Address              Cost
        A             250 West 105th St.   $2,485
    B             170 West 109th St.   $2,750


In [50]:
# display map
benchmark_map

In [51]:
apt_A_map

In [52]:
apt_b_map


---
## 5. Discussion
##### 5.1 Discusssion, observations and recommendation for improvements
Our results all show the cost, and proximity to restaurants and subway stops.
Taking this into account, both apartment A and apartment B will fullfill all of our problem statements. Now we'll compare them to the benchmark scenario.
We see that apartment A and B both have chinese food, and subway stops closer than our benchmark. This is a good scenario to have. Since they both are better than our current situation, we don't rule out either apartment based on this. If an apartment we were looking at had locations worse than our benchmark, but still satisfying the problem criteria, we could rule it out based on our preferences.

Our final decision now needs to take into account which of the two apartments we will choose. Aparment A is cheaper than apartment B, and is closer to a subway stop and Chinese food restaurant. Because apartment A has closer ammenities, and is cheaper, this is the prefered apartment. Our final choice for which apartment we will chose is apartment A.



---
## 6. Conclusion
##### 6.1 Decision taken & report conclusion
This project set out to find an apartment in New York that is cheaper than $3,000, close to a subway stop, and close to certain restaurants. We used Foursquare search and explore mechanisms to find which neighborhood we would like to live in based on what is in each neighborhood. Based on preference we were able to search in a more restricted area, and limit the number of availible and open units. Finding two apartments that fit our price criteria, we then wanted to see if they both fit the proximity restrictions. Using Foursquare search functions, we found nearby locations that satisfy the criteria. We finally chose an apartment based on which apartment was cheaper and closer to a subway and restaurant.

Python coded Jupyter books with a link to Foursquare API is a very useful tool that can be used to find all sorts of information. We used it to find an apartment, but it is useful for other purposes. 

